# 反爬：登入授權模擬

* 了解「登入權限機制」的反爬蟲機制
* 「登入權限機制」反爬蟲的因應策略

## 作業目標

* 找一個需要登入的網站試試看，並說明思考流程
（如果不知道要用哪個網站的話，可以試試看 https://github.com/new 網址，未登入時會被導向登入頁）



In [ ]:
'''
Your Code
'''

In [1]:
from lxml import html
import requests
from bs4 import BeautifulSoup

requests.packages.urllib3.disable_warnings()

In [2]:
#This URL will be the URL that your login form points to with the "action" tag.
URL_POST_LOGIN = 'https://github.com/login'
#This URL is the page you actually want to pull down with requests.
URL_REQUEST = 'https://github.com/new'

## 未授權前存取

In [3]:
res = requests.get('https://github.com/new', verify=False)
len(res.text)

15365

## 模仿授權機制

### 方法一：先送登入，再送請求

In [ ]:
with requests.Session() as sess:
    ##
    r = sess.get(URL_REQUEST)
    tree = html.fromstring(r.text)
    authenticity_token = tree.xpath("//input[@name='authenticity_token']/@value")[0]
    timestamp = tree.xpath("//input[@name='timestamp']/@value")[0]
    timestamp_secret = tree.xpath("//input[@name='timestamp_secret']/@value")[0]
    
    
    # login information
    payload = {
        'authenticity_token': authenticity_token,
        'login': "YOUR EMAIL OR USENAME", # <------------------------------ [!]
        'password': "YOUR PASSWORD", # <----------------------------------- [!]
        "timestamp": timestamp,
        "timestamp_secret": timestamp_secret,
        'commit': 'Sign in',
    }
    
    ##
    post = sess.post(URL_POST_LOGIN, data=payload)
    r = sess.get(URL_REQUEST)
    #print(r.text)
    print(len(r.text))

### 方法二：在請求中帶入 cookie

In [4]:
cookies = {'user_session': 'YSx17YxKc2EEmpsQKNHzCPzqfffcGxBMWFsPub5M7nfVXJLS'}
res = requests.get('https://github.com/new', verify=False, cookies=cookies)
len(res.text)

140582

In [5]:
soup = BeautifulSoup(res.text,'html.parser')
soup.title

<title>Create a New Repository</title>